In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sksurgeryimage.calibration.chessboard_point_detector as cpd
import sksurgerycalibration.video.video_calibration_driver_mono as mc

# Calibration

In [ ]:
chessboard_corners = (14, 10)
min_points_to_detect = chessboard_corners[0] * chessboard_corners[1]
square_size_mm = 6

video_source = cv2.VideoCapture(1)
video_source.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
video_source.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
video_source.set(cv2.CAP_PROP_AUTOFOCUS, 0)
video_source.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)

You may re-run the below cell to clear obj/img points.

In [ ]:
detector = cpd.ChessboardPointDetector(chessboard_corners, square_size_mm)
calibrator = mc.MonoVideoCalibrationDriver(detector, min_points_to_detect)

In [ ]:
_, frame = video_source.read()
number_of_points = calibrator.grab_data(frame)

if number_of_points > 0:
    img_pts = calibrator.video_data.image_points_arrays[-1]
    ###print(img_pts.shape)##(140, 1, 2)
    frame = cv2.drawChessboardCorners(
        frame, chessboard_corners, img_pts, number_of_points
    )
plt.imshow(frame)
plt.show()

number_of_views = calibrator.get_number_of_views()
print(f"Detected {number_of_points} corners")
print(f"{number_of_views} frames captured")

If detected output is undesirable, pop that set of obj/img points in the cell below. Note that `pop()` will only work when views > 1.

In [ ]:
calibrator.pop()
calibrator.get_number_of_views()

Get intrinsics. Ideally, `rms < 0.5`.

In [ ]:
if number_of_views > 1:
    rms, params = calibrator.calibrate()
    print(f"Reprojection (2D) error is: \n {rms}")
    print(f"Intrinsics are: \n  {params.camera_matrix.tolist()}")
    print(f"Distortion matrix is:  \n {params.dist_coeffs.tolist()}")

In [ ]:
video_source.release()

# Undistortion

In [ ]:
h, w = frame.shape[:2]
newintrinsics, roi = cv2.getOptimalNewCameraMatrix(
    params.camera_matrix, params.dist_coeffs, (w, h), 1, (w, h)
)

In [ ]:
plt.imshow(frame)

In [ ]:
dst = cv2.undistort(frame, params.camera_matrix, params.dist_coeffs)

# Only run this if you want to crop the frame.
# x, y, w, h = roi
# dst = dst[y : y + h, x : x + w]

plt.imshow(dst)